Будем практиковаться на датасете: https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:

    Создать Dataset для загрузки данных (используем только числовые данные)
    Обернуть его в Dataloader
    Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
    Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
    Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [ ]:
import math
import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train_path = '/content/gdrive/MyDrive/pytourch/lesson3_torch/train.csv'

In [ ]:
train_df = pd.read_csv(train_path)

In [ ]:
train_df

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503419,5e6b7f0f3f65,dd05217165ea,Пермский край,Пермь,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,26,Ботиночки,NaN,300.0,7,2017-03-20,Private,2da79294c6f23f61d4bf6a8f4034f87e2f318b7e32c68b...,88.0,0.25019
1503420,d1f0910d2126,dc23ef451c8a,Краснодарский край,Краснодар,Услуги,Предложение услуг,"Ремонт, строительство",Ремонт квартиры,NaN,Натяжной потолок,"Произведем бесплатный замер, изготовим и устан...",200.0,1,2017-03-28,Private,33e13a67b441486ebe48e1459382caa620c4d1802aad1c...,1191.0,0.60000
1503421,bc04866bc803,66a7016a5995,Башкортостан,Белебей,Недвижимость,Квартиры,Продам,4,Вторичка,"4-к квартира, 83 м², 4/5 эт.","Продам квартиру с отличным ремонтом , две лодж...",NaN,50,2017-03-21,Company,2ae822ca21cff9465e430c68a1df31c5cda658cbc00eb1...,1202.0,0.39569
1503422,f782f2ad9349,2d3ac1f5a7b3,Пермский край,Березники,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,27,Туфли-балетки,Продам красивые туфли в хорошем состоянии!,350.0,33,2017-03-22,Private,6b4285c22388a5dae37c3d390d3e1b174f7a980d9e3919...,87.0,0.00000


In [ ]:
train_df.describe()

,price,item_seq_number,image_top_1,deal_probability
count,1.418062e+06,1.503424e+06,1.390836e+06,1.503424e+06
mean,3.167081e+05,7.436740e+02,1.241932e+03,1.391306e-01
std,6.689154e+07,5.572522e+03,9.704641e+02,2.600785e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+02,9.000000e+00,4.250000e+02,0.000000e+00
50%,1.300000e+03,2.900000e+01,1.057000e+03,0.000000e+00
75%,7.000000e+03,8.800000e+01,2.217000e+03,1.508700e-01
max,7.950101e+10,2.044290e+05,3.066000e+03,1.000000e+00


In [ ]:
train_df.loc[0, ['price', 'item_seq_number', 'image_top_1', 'deal_probability']]

price                 400.0
item_seq_number           2
image_top_1          1008.0
deal_probability    0.12789
Name: 0, dtype: object

In [ ]:
numerical_features = train_df.select_dtypes(include=[np.number])
print(f"count of numeric_features {numerical_features.shape[1]}")

numerical_features.columns

count of numeric_features 4


Index(['price', 'item_seq_number', 'image_top_1', 'deal_probability'], dtype='object')

In [ ]:
len(train_df)

1503424

# Новый раздел

In [ ]:
class AvitoDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file):
        self.data_frame = pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, index):
        features = []
        target = []
        price = self.data_frame.loc[index, 'price']
        features.append(price)
        item_seq_number = self.data_frame.loc[index, 'item_seq_number']
        features.append(item_seq_number)
        image_top_1 = self.data_frame.loc[index, 'image_top_1']
        features.append(image_top_1)
        deal_probability = self.data_frame.loc[index, 'deal_probability']
        target.append(deal_probability)
        #features = {'price': price, 'item_seq_number': item_seq_number, 'image_top_1': image_top_1}
        #target = {'deal_probability': deal_probability}
        features = np.asarray(features)
        target = np.asarray(target)
        return features, target

In [ ]:
avito_train_dataset = AvitoDataset(train_path)

In [ ]:
avito_train_dataset[0]

(array([ 400.,    2., 1008.]), array([0.12789]))

In [ ]:
train_loader = torch.utils.data.DataLoader(avito_train_dataset, batch_size=4, shuffle=False, num_workers=2)

In [ ]:
train_loader

In [ ]:
class Perceptron(nn.Module):
  def __init__(self, input_dim, output_dim, activation="relu"):
    super(Perceptron, self).__init__()
    self.fc = nn.Linear(input_dim, output_dim)
    self.activation = activation

  def forward(self, x):
    x = self.fc(x)
    if self.activation=="relu":
      return F.relu(x)
    if self.activation=="sigmoid":
      return F.sigmoid(x)
    raise RuntimeError


class FeedForward(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super(FeedForward, self).__init__()
    self.bn1 = nn.BatchNorm1d(input_dim)
    self.fc1 = Perceptron(input_dim, hidden_dim)
    self.bn2 = nn.BatchNorm1d(hidden_dim)
    self.dp = nn.Dropout(0.25)
    self.fc2 = Perceptron(hidden_dim, 10, "relu")

  def forward(self, x):
    x = self.bn1(x)
    x = self.fc1(x)
    x = self.bn2(x)
    x = self.dp(x)
    x = self.fc2(x)
    return x

In [ ]:
net = FeedForward(3, 200)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in tqdm(range(10)):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    inputs, labels = data[0], data[1]
    print(inputs.shape)
    print(inputs)
    # обнуляем градиент
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # выводим статистику о процессе обучения
    running_loss += loss.item()
    if i % 300 == 0:
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 300))
      running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([4, 3])
tensor([[4.0000e+02, 2.0000e+00, 1.0080e+03],
        [3.0000e+03, 1.9000e+01, 6.9200e+02],
        [4.0000e+03, 9.0000e+00, 3.0320e+03],
        [2.2000e+03, 2.8600e+02, 7.9600e+02]], dtype=torch.float64)


RuntimeError: ignored